In [229]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [230]:
jester_data = pd.read_csv('jester-data-1.csv', header = None)
jester_data = jester_data.drop([0],axis = 1)

In [231]:
jester_data.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [232]:
jester_dataCopy = jester_data.copy()
jester_dataCopy.head()
jester_dataCopy.shape

(24983, 100)

In [234]:
original = jester_dataCopy.values.copy()
original2 = jester_dataCopy.values.copy()
#print(jester_dataCopy.values)
train = np.full((jester_dataCopy.shape[0],jester_dataCopy.shape[1]),99.00)
train

count = 0
for i in range(0,jester_dataCopy.shape[0]):
    for j in range(0, jester_dataCopy.shape[1]):
        if(original[i,j] != 99):
            if(np.random.random() > 0.1):
                train[i,j] = original[i,j]
                original[i,j] = 99
        
#print('This is the training', train)
#print('This is the validation',original)
        

In [235]:
n_features = 2

user_ratings = train[:100,:]
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

In [236]:
latent_item_features
latent_user_preferences

array([[0.151, 0.199],
       [0.949, 0.382],
       [0.429, 0.456],
       [0.515, 0.931],
       [0.208, 0.41 ],
       [0.431, 0.872],
       [0.682, 0.245],
       [0.303, 0.922],
       [0.911, 0.903],
       [0.76 , 0.407],
       [0.819, 0.471],
       [0.194, 0.272],
       [0.413, 0.009],
       [0.863, 0.217],
       [0.238, 0.12 ],
       [0.456, 0.96 ],
       [0.118, 0.543],
       [0.06 , 0.728],
       [0.656, 0.433],
       [0.179, 0.075],
       [0.377, 0.832],
       [0.759, 0.802],
       [0.006, 0.136],
       [0.995, 0.161],
       [0.266, 0.248],
       [0.544, 0.568],
       [0.926, 0.726],
       [0.276, 0.499],
       [0.06 , 0.853],
       [0.671, 0.122],
       [0.284, 0.556],
       [0.652, 0.08 ],
       [0.839, 0.569],
       [0.038, 0.234],
       [0.643, 0.568],
       [0.845, 0.367],
       [0.73 , 0.178],
       [0.044, 0.061],
       [0.265, 0.279],
       [0.627, 0.054],
       [0.239, 0.367],
       [0.634, 0.138],
       [0.912, 0.952],
       [0.5

In [237]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 1000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if(rating != 99):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if(iteration%100 == 0 ):
            print(mse)

In [238]:
sgd()

27.46206017990744
16.84708606402183
16.84328248781642
16.846676625061285
16.851859834149206
16.85888370028112
16.86779882364773
16.878676603404568
16.891610262810964
16.906714193662854


In [239]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[-6.257, -3.644, -7.404, ...,  1.508, -1.58 , -0.993],
       [ 4.1  ,  2.974,  4.555, ...,  0.332,  1.473,  1.869],
       [ 9.038,  8.104,  9.257, ...,  4.217,  4.401,  7.339],
       ...,
       [-0.567, -0.396, -0.637, ..., -0.012, -0.193, -0.227],
       [ 0.691,  1.016,  0.506, ...,  1.217,  0.633,  1.387],
       [-4.233, -3.447, -4.512, ..., -1.19 , -1.801, -2.712]])

In [242]:
values = [zip(user_ratings[i], predictions[i]) for i in range(0,predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = jester_data.columns
# comparison_data.applymap(lambda (x,y): "(%2.3f|%2.3f)"%(x,y))

In [243]:
comparison_data

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,"(99.0, -6.256938959802785)","(99.0, -3.6444982317126273)","(99.0, -7.403616963740385)","(99.0, -10.593175174598656)","(99.0, -6.396789571114272)","(99.0, -0.6020658747360379)","(99.0, -3.6383144392515345)","(99.0, -6.098006968607771)","(99.0, -10.818888910200165)","(99.0, -7.102874583922789)",...,"(99.0, 0.03817166663081162)","(99.0, -1.274772555670281)","(99.0, 2.0211524234585334)","(99.0, -4.995421909841947)","(99.0, -3.1794256305088666)","(99.0, -0.3990106648992162)","(99.0, -3.248609671691755)","(99.0, 1.5075698322201934)","(99.0, -1.580467891696442)","(99.0, -0.9928749957692671)"
1,"(99.0, 4.100420485721645)","(99.0, 2.9741332477555766)","(99.0, 4.555165611303221)","(4.37, 5.614567962967988)","(99.0, 3.7593136379550294)","(99.0, 2.063214009974126)","(99.0, 2.6902856850067556)","(99.0, 3.1785177644843827)","(99.0, 5.682723691132364)","(99.0, 4.385084539302476)",...,"(99.0, 1.223636321084348)","(99.0, 2.0480908198195937)","(99.0, 0.8937803767065926)","(99.0, 3.224375070855986)","(99.0, 2.7950207931537117)","(99.0, 1.4057628354463128)","(3.06, 2.5659758384140092)","(99.0, 0.331502715861145)","(99.0, 1.47270219338545)","(99.0, 1.8688544029930085)"
2,"(99.0, 9.03838096547986)","(99.0, 8.103608860475452)","(99.0, 9.256670877471828)","(99.0, 8.867843582111401)","(99.0, 7.142925014610822)","(99.0, 8.957315895865946)","(99.0, 6.738571092926407)","(99.0, 4.845352201620202)","(99.0, 8.80729964446795)","(99.0, 8.953135989859803)",...,"(99.0, 5.996799217482662)","(99.0, 7.719057445469954)","(99.0, 7.8320867817135635)","(99.0, 6.977013945540605)","(99.0, 8.040879686223242)","(99.0, 6.122435844649498)","(99.0, 6.810939842561267)","(99.0, 4.217454373541934)","(99.0, 4.400887797505294)","(99.0, 7.33851410647046)"
3,"(99.0, 2.1991184630702314)","(99.0, 2.6023588689078068)","(99.0, 1.9327520445661373)","(99.0, 0.7282343829455002)","(99.0, 1.2719860635194256)","(99.0, 3.9760453083734575)","(-2.82, 1.9689762699358986)","(99.0, 0.29844358337001275)","(99.0, 0.6276218031570453)","(99.0, 1.8879782552531255)",...,"(99.0, 2.8035041901574944)","(99.0, 3.183815883275421)","(99.0, 4.294094623756331)","(99.0, 1.6444629843358378)","(99.0, 2.722403761904287)","(99.0, 2.721690078255114)","(99.0, 2.1277178205950724)","(99.0, 2.446827465209229)","(99.0, 1.5412509261242753)","(99.0, 3.097150599652978)"
4,"(99.0, 2.970125570705251)","(99.0, 3.5298722689030306)","(99.0, 2.602706796965764)","(99.0, 0.9492589113100145)","(99.0, 1.706763315003516)","(99.0, 5.413144158831822)","(99.0, 2.6671992413194268)","(99.0, 0.3819535308924811)","(99.0, 0.8113113082559456)","(99.0, 2.542933041564612)",...,"(99.0, 3.818662718075948)","(99.0, 4.331381862607371)","(99.0, 5.856899821104979)","(99.0, 2.2197347420157247)","(99.0, 3.695250916693685)","(1.55, 3.7054684971789666)","(99.0, 2.884981305060564)","(99.0, 3.338765191746367)","(99.0, 2.0928982230586506)","(99.0, 4.214474013816063)"
5,"(99.0, -6.260068933082407)","(99.0, -5.189611042984261)","(99.0, -6.625547674015007)","(99.0, -7.100712520181482)","(99.0, -5.259793294322593)","(99.0, -4.998824673530723)","(99.0, -4.446239889420652)","(-0.87, -3.9465121547626385)","(99.0, -7.1163840927887065)","(-1.8, -6.395811900281357)",...,"(99.0, -3.2516684354708447)","(99.0, -4.4704983121989)","(99.0, -3.822513765732591)","(99.0, -4.867967511304235)","(99.0, -5.055407164565997)","(99.0, -3.4140658528379975)","(99.0, -4.401698263302757)","(99.0, -1.9681302827053426)","(99.0, -2.732526737488556)","(-3.45, -4.202956660990746)"
6,"(99.0, 4.642402992549558)","(99.0, 4.335874760317656)","(99.0, 4.666584046902671)","(99.0, 4.161355100660785)","(99.0, 3.540578090321266)","(99.0, 5.0953118068944985)","(99.0, 3.551823484529017)","(99.0, 2.2463692692276034)","(99.0, 4.1066179734205805)","(99.0, 4.518683049041257)",...,"(99.0, 3.450216809995465)","(99.0, 4.324163429981521)","(99.0, 4.680264541664214)","(99.0, 3.5690005103606395)","(99.0, 4.340899798979102)","(99.0, 3.4838133569629885)","(9

In [244]:
values = [zip(original[i], predictions[i]) for i in range(0,predictions.shape[0])]
comparison_data2 = pd.DataFrame(values)
comparison_data2.columns = jester_data.columns

In [245]:
comparison_data2

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,"(99.0, -6.256938959802785)","(99.0, -3.6444982317126273)","(99.0, -7.403616963740385)","(99.0, -10.593175174598656)","(99.0, -6.396789571114272)","(99.0, -0.6020658747360379)","(99.0, -3.6383144392515345)","(99.0, -6.098006968607771)","(99.0, -10.818888910200165)","(99.0, -7.102874583922789)",...,"(99.0, 0.03817166663081162)","(99.0, -1.274772555670281)","(99.0, 2.0211524234585334)","(99.0, -4.995421909841947)","(99.0, -3.1794256305088666)","(99.0, -0.3990106648992162)","(99.0, -3.248609671691755)","(99.0, 1.5075698322201934)","(99.0, -1.580467891696442)","(99.0, -0.9928749957692671)"
1,"(99.0, 4.100420485721645)","(99.0, 2.9741332477555766)","(99.0, 4.555165611303221)","(4.37, 5.614567962967988)","(99.0, 3.7593136379550294)","(99.0, 2.063214009974126)","(99.0, 2.6902856850067556)","(99.0, 3.1785177644843827)","(99.0, 5.682723691132364)","(99.0, 4.385084539302476)",...,"(99.0, 1.223636321084348)","(99.0, 2.0480908198195937)","(99.0, 0.8937803767065926)","(99.0, 3.224375070855986)","(99.0, 2.7950207931537117)","(99.0, 1.4057628354463128)","(3.06, 2.5659758384140092)","(99.0, 0.331502715861145)","(99.0, 1.47270219338545)","(99.0, 1.8688544029930085)"
2,"(99.0, 9.03838096547986)","(99.0, 8.103608860475452)","(99.0, 9.256670877471828)","(99.0, 8.867843582111401)","(99.0, 7.142925014610822)","(99.0, 8.957315895865946)","(99.0, 6.738571092926407)","(99.0, 4.845352201620202)","(99.0, 8.80729964446795)","(99.0, 8.953135989859803)",...,"(99.0, 5.996799217482662)","(99.0, 7.719057445469954)","(99.0, 7.8320867817135635)","(99.0, 6.977013945540605)","(99.0, 8.040879686223242)","(99.0, 6.122435844649498)","(99.0, 6.810939842561267)","(99.0, 4.217454373541934)","(99.0, 4.400887797505294)","(99.0, 7.33851410647046)"
3,"(99.0, 2.1991184630702314)","(99.0, 2.6023588689078068)","(99.0, 1.9327520445661373)","(99.0, 0.7282343829455002)","(99.0, 1.2719860635194256)","(99.0, 3.9760453083734575)","(-2.82, 1.9689762699358986)","(99.0, 0.29844358337001275)","(99.0, 0.6276218031570453)","(99.0, 1.8879782552531255)",...,"(99.0, 2.8035041901574944)","(99.0, 3.183815883275421)","(99.0, 4.294094623756331)","(99.0, 1.6444629843358378)","(99.0, 2.722403761904287)","(99.0, 2.721690078255114)","(99.0, 2.1277178205950724)","(99.0, 2.446827465209229)","(99.0, 1.5412509261242753)","(99.0, 3.097150599652978)"
4,"(99.0, 2.970125570705251)","(99.0, 3.5298722689030306)","(99.0, 2.602706796965764)","(99.0, 0.9492589113100145)","(99.0, 1.706763315003516)","(99.0, 5.413144158831822)","(99.0, 2.6671992413194268)","(99.0, 0.3819535308924811)","(99.0, 0.8113113082559456)","(99.0, 2.542933041564612)",...,"(99.0, 3.818662718075948)","(99.0, 4.331381862607371)","(99.0, 5.856899821104979)","(99.0, 2.2197347420157247)","(99.0, 3.695250916693685)","(1.55, 3.7054684971789666)","(99.0, 2.884981305060564)","(99.0, 3.338765191746367)","(99.0, 2.0928982230586506)","(99.0, 4.214474013816063)"
5,"(99.0, -6.260068933082407)","(99.0, -5.189611042984261)","(99.0, -6.625547674015007)","(99.0, -7.100712520181482)","(99.0, -5.259793294322593)","(99.0, -4.998824673530723)","(99.0, -4.446239889420652)","(-0.87, -3.9465121547626385)","(99.0, -7.1163840927887065)","(-1.8, -6.395811900281357)",...,"(99.0, -3.2516684354708447)","(99.0, -4.4704983121989)","(99.0, -3.822513765732591)","(99.0, -4.867967511304235)","(99.0, -5.055407164565997)","(99.0, -3.4140658528379975)","(99.0, -4.401698263302757)","(99.0, -1.9681302827053426)","(99.0, -2.732526737488556)","(-3.45, -4.202956660990746)"
6,"(99.0, 4.642402992549558)","(99.0, 4.335874760317656)","(99.0, 4.666584046902671)","(99.0, 4.161355100660785)","(99.0, 3.540578090321266)","(99.0, 5.0953118068944985)","(99.0, 3.551823484529017)","(99.0, 2.2463692692276034)","(99.0, 4.1066179734205805)","(99.0, 4.518683049041257)",...,"(99.0, 3.450216809995465)","(99.0, 4.324163429981521)","(99.0, 4.680264541664214)","(99.0, 3.5690005103606395)","(99.0, 4.340899798979102)","(99.0, 3.4838133569629885)","(9